In [1]:
import numpy as np
import pandas as pd

df = pd.read_excel("../data/underlying_assets.xlsx", engine="openpyxl")

starting_date = "2017/06/30"
checking_date = np.array(["2017/12/21", "2018/06/26", "2018/12/20", "2019/06/25", "2019/12/19", "2020/06/25"])
strike_price = np.array([0.87, 0.87, 0.87, 0.85, 0.85, 0.85])
knock_in_barrier = np.array([0.55])

In [2]:
first_star_raw = df[df.Name == "삼성전자"].set_index("Date").loc[:, "Close"]
second_star_raw = df[df.Name == "현대차"].set_index("Date").loc[:, "Close"]

first_star_s0 = first_star_raw[starting_date]
second_star_s0 = second_star_raw[starting_date]
s0 = np.array([first_star_s0, second_star_s0])

first_star_at_checking_date = first_star_raw[checking_date].values
second_star_at_checking_date = second_star_raw[checking_date].values
episode_at_checking_date = np.vstack([first_star_at_checking_date, second_star_at_checking_date])

In [3]:
success_event = np.array([0, 0, 0, 0, 0, 0, 0])
failure_event = np.array([0])

performance = np.divide(episode_at_checking_date, np.repeat(np.array([first_star_s0, second_star_s0]), len(strike_price)).reshape(2, -1))
worst_performance = np.min(performance, axis=0)
worst_performance_index = np.where(performance == worst_performance)[0]

for i in range(len(strike_price)):
    if worst_performance[i] >= strike_price[i]:
        success_event[i] += 1
        is_payoff = True
        break

if worst_performance[i] > knock_in_barrier and is_payoff == False:
    success_event[-1] += 1
elif worst_performance[i] <= knock_in_barrier and is_payoff == False:
    failure_event += 1
    
print(f"Sucess event: {success_event}")
print(f"Failure event: {failure_event}")
    

Sucess event: [1 0 0 0 0 0 0]
Failure event: [0]
